# Creating the json for kaggle api

The data set is a set of data from twitter that can classify the sentiment of a tweet regarding Covid,
The data set has been manually tagged to ensure quality of the set. The set has been predivided into train and test.
This set contains 2 files,
1. Corona_NLP_test.csv
2. Corona_NLP_train.csv

both files consists of 6 columns,
1. UserName
2. ScreenName
3. Location
4. TweetAt
5. OriginalTweet
6. Sentiment

Our notebook here uses a Sequential Model, RNN using GRU and Finally using Embedding.
Using these models we have trained it to be able to predict the sentiment of a tweet related to covid

In [ ]:
from os import mkdir

!pip install kaggle
!mkdir /root/.kaggle
!ls -la

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
total 15740
drwxr-xr-x 1 root root     4096 Nov 22 16:47 .
drwxr-xr-x 1 root root     4096 Nov 22 16:46 ..
drwxr-xr-x 4 root root     4096 Nov 18 14:33 .config
-rw-r--r-- 1 root root  1002494 Sep  8  2020 Corona_NLP_test.csv
-rw-r--r-- 1 root root 10500262 Sep  8  2020 Corona_NLP_train.csv
-rw-r--r-- 1 root root  4595012 Nov 22 16:47 covid-19-nlp-text-classification.zip
drwxr-xr-x 1 root root     4096 Nov 18 14:34 sample_data


In [ ]:
import json
kag_json = {"username":"shararsiddiqui","key":"0030c69a6996d6d07e9727567cf64046"}

#serialize
json_obj = json.dumps(kag_json)
with open("/root/.kaggle/kaggle.json", "w") as outfile:
    outfile.write(json_obj)

!ls .kaggle/
!cat ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d datatattle/covid-19-nlp-text-classification
!unzip covid-19-nlp-text-classification

# search - https://www.kaggle.com/search?q=corona+virus+tweets+in%3Adatasets
# dataset chosen - https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification

ls: cannot access '.kaggle/': No such file or directory
{"username": "shararsiddiqui", "key": "0030c69a6996d6d07e9727567cf64046"}covid-19-nlp-text-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  covid-19-nlp-text-classification.zip
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Corona_NLP_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Call kaggle api to download the required datasets

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

# Sequential Model

# Explain what the data is supposed to help with

In [ ]:
# the data we obtained was already divided into train and test
df_test = pd.read_csv('Corona_NLP_test.csv', header=0, usecols=[4,5], encoding='latin-1')
df_train = pd.read_csv('Corona_NLP_train.csv', header=0, usecols=[4,5], encoding='latin-1')[:16000]

df_test.groupby(['Sentiment'])['Sentiment'].count()
df_train.groupby(['Sentiment'])['Sentiment'].count()


Sentiment
Extremely Negative    2464
Extremely Positive    2423
Negative              4014
Neutral               2813
Positive              4286
Name: Sentiment, dtype: int64

Vectorize each tweet:

In [ ]:
# set up X and Y
num_labels = 5
vocab_size = 25000
batch_size = 100

# fit the tokenizer on the training data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df_train.OriginalTweet)

x_train = tokenizer.texts_to_matrix(df_train.OriginalTweet, mode='tfidf')
x_test = tokenizer.texts_to_matrix(df_test.OriginalTweet, mode='tfidf')

encoder = LabelEncoder()
encoder.fit(df_train.Sentiment)
y_train = encoder.transform(df_train.Sentiment)
y_test = encoder.transform(df_test.Sentiment)

# check shape
print("train shapes:", x_train.shape, y_train.shape)
print("test shapes:", x_test.shape, y_test.shape)
print("test first five labels:", y_test[:5])

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)
y_test = to_categorical(y_test, num_classes=None)

train shapes: (16000, 25000) (16000,)
test shapes: (3798, 25000) (3798,)
test first five labels: [0 4 1 2 3 3 4 3 0 1]


Train a sequential model

In [ ]:
models.Sequential()
seq_model.add(layers.Dense(24, input_dim=vocab_size, kernel_initializer='normal', activation='relu'))
seq_model.add(layers.Dropout(.35))
seq_model.add(layers.Dense(12, kernel_initializer='normal', activation='relu'))
seq_model.add(layers.Dropout(.35))
seq_model.add(layers.Dense(5, kernel_initializer='normal', activation='softmax'))


seq_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = seq_model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=7,
                    verbose=1,
                    validation_split=0.1)




Epoch 1/7
144/144 [==============================] - 3s 13ms/step - loss: 1.5797 - accuracy: 0.2662 - val_loss: 1.5479 - val_accuracy: 0.2931
Epoch 2/7
144/144 [==============================] - 2s 12ms/step - loss: 1.4052 - accuracy: 0.3708 - val_loss: 1.2925 - val_accuracy: 0.4656
Epoch 3/7
144/144 [==============================] - 2s 12ms/step - loss: 1.1261 - accuracy: 0.5253 - val_loss: 1.1496 - val_accuracy: 0.5356
Epoch 4/7
144/144 [==============================] - 2s 11ms/step - loss: 0.9001 - accuracy: 0.6422 - val_loss: 1.1277 - val_accuracy: 0.5394
Epoch 5/7
144/144 [==============================] - 2s 11ms/step - loss: 0.7387 - accuracy: 0.7233 - val_loss: 1.1682 - val_accuracy: 0.5400
Epoch 6/7
144/144 [==============================] - 2s 12ms/step - loss: 0.5973 - accuracy: 0.7871 - val_loss: 1.2434 - val_accuracy: 0.5456
Epoch 7/7
144/144 [==============================] - 2s 12ms/step - loss: 0.5050 - accuracy: 0.8226 - val_loss: 1.4017 - val_accuracy: 0.5381


In [ ]:
# evaluate
score = seq_model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

38/38 [==============================] - 0s 5ms/step - loss: 1.4496 - accuracy: 0.5300
Accuracy:  0.5300158262252808


# 3 Try a different architecture like RNN, CNN, etc and evaluate on the test data

## GRU (RNN) Architecture
Here, we train a GRU without any embedding layer. Instead, we use a one-hot encoding with a vocab size of 500.

Vectorize the words of each tweet:

In [ ]:
LSTM_VOCAB_SIZE = 500

maxlen = 500
batch_size = 32

from tensorflow import keras


encoder = LabelEncoder()
encoder.fit(df_train.Sentiment)
y_train = encoder.transform(df_train.Sentiment)
y_test = encoder.transform(df_test.Sentiment)

encoder = keras.layers.TextVectorization(max_tokens=LSTM_VOCAB_SIZE)
encoder.adapt(df_train.OriginalTweet)

x_train = encoder(df_train.OriginalTweet)
x_test = encoder(df_test.OriginalTweet)

x_train = tf.keras.utils.to_categorical(x_train, num_classes = LSTM_VOCAB_SIZE)
x_test = tf.keras.utils.to_categorical(x_test, num_classes = LSTM_VOCAB_SIZE)

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)
y_test = to_categorical(y_test, num_classes=None)


Train the gru:

In [ ]:
lstm = keras.Sequential()
lstm.add(layers.Bidirectional(layers.GRU(48, input_dim=LSTM_VOCAB_SIZE)))
lstm.add(layers.Dense(10, activation='relu'))
lstm.add(layers.Dense(5, activation='softmax'))

lstm.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = lstm.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=7,
                    verbose=1,
                    validation_split=0.1)



Epoch 1/7
450/450 [==============================] - 50s 101ms/step - loss: 1.5280 - accuracy: 0.2989 - val_loss: 1.4500 - val_accuracy: 0.3519
Epoch 2/7
450/450 [==============================] - 45s 100ms/step - loss: 1.3640 - accuracy: 0.4021 - val_loss: 1.2525 - val_accuracy: 0.4806
Epoch 3/7
450/450 [==============================] - 46s 103ms/step - loss: 1.2222 - accuracy: 0.4806 - val_loss: 1.2109 - val_accuracy: 0.4994
Epoch 4/7
450/450 [==============================] - 47s 105ms/step - loss: 1.1795 - accuracy: 0.5063 - val_loss: 1.2539 - val_accuracy: 0.4956
Epoch 5/7
450/450 [==============================] - 45s 100ms/step - loss: 1.1542 - accuracy: 0.5164 - val_loss: 1.2360 - val_accuracy: 0.4863
Epoch 6/7
450/450 [==============================] - 46s 102ms/step - loss: 1.1291 - accuracy: 0.5282 - val_loss: 1.2316 - val_accuracy: 0.4844
Epoch 7/7
450/450 [==============================] - 45s 100ms/step - loss: 1.1069 - accuracy: 0.5424 - val_loss: 1.2228 - val_accuracy:

In [ ]:
# evaluate
score = lstm.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

119/119 [==============================] - 6s 36ms/step - loss: 1.2584 - accuracy: 0.4813
Accuracy:  0.48130595684051514


# Using an Embedding

Now, we'll apply an embedding layer. The vocab size is 10000 here

In [ ]:
LSTM_VOCAB_SIZE = 10000

maxlen = 500
batch_size = 32

from tensorflow import keras

encoder = LabelEncoder()
encoder.fit(df_train.Sentiment)
y_train = encoder.transform(df_train.Sentiment)
y_test = encoder.transform(df_test.Sentiment)

encoder = keras.layers.TextVectorization(max_tokens=LSTM_VOCAB_SIZE)
encoder.adapt(df_train.OriginalTweet)

x_train = encoder(df_train.OriginalTweet)
x_test = encoder(df_test.OriginalTweet)

from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=None)
y_test = to_categorical(y_test, num_classes=None)


In [ ]:
lstm = keras.Sequential()
lstm.add(layers.Embedding(input_dim=LSTM_VOCAB_SIZE, output_dim=64))
lstm.add(layers.Bidirectional(layers.GRU(64, input_dim=LSTM_VOCAB_SIZE)))
lstm.add(layers.Dense(10, activation='relu'))
lstm.add(layers.Dense(5, activation='softmax'))

lstm.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = lstm.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=7,
                    verbose=1,
                    validation_split=0.1)


Epoch 1/7
450/450 [==============================] - 33s 65ms/step - loss: 1.4513 - accuracy: 0.3368 - val_loss: 1.2816 - val_accuracy: 0.4412
Epoch 2/7
450/450 [==============================] - 27s 61ms/step - loss: 0.9409 - accuracy: 0.6203 - val_loss: 0.8421 - val_accuracy: 0.6712
Epoch 3/7
450/450 [==============================] - 32s 72ms/step - loss: 0.6459 - accuracy: 0.7655 - val_loss: 0.8091 - val_accuracy: 0.7063
Epoch 4/7
450/450 [==============================] - 27s 61ms/step - loss: 0.4968 - accuracy: 0.8285 - val_loss: 0.9122 - val_accuracy: 0.6800
Epoch 5/7
450/450 [==============================] - 31s 68ms/step - loss: 0.3778 - accuracy: 0.8742 - val_loss: 0.9791 - val_accuracy: 0.6787
Epoch 6/7
450/450 [==============================] - 27s 61ms/step - loss: 0.2821 - accuracy: 0.9119 - val_loss: 1.1017 - val_accuracy: 0.6669
Epoch 7/7
450/450 [==============================] - 27s 61ms/step - loss: 0.2145 - accuracy: 0.9349 - val_loss: 1.2213 - val_accuracy: 0.6712

In [ ]:
# evaluate
score = lstm.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

119/119 [==============================] - 3s 16ms/step - loss: 1.3386 - accuracy: 0.6477
Accuracy:  0.647709310054779


# Performance Evaluation
## Testing Accuracies
Below you can see the testing accuracies of each model:

*   Basic Sequential: `53%`
*   RNN/Gru Architecture (without embedding): `48.1%`
*   RNN/Gru Architecture (with embedding layer): `64.77%`

NOTE: While these accuracies may seem low, it should be noted that it is a multi-category classification task. An untrained model should only get an accuracy of 1/5 = 20% on average, so these accuracies are relatively high.

## Time Complexity
The sequential model trained and evaluated quickly, while the RNN architectures were slow to evaluate and train.

## Comparison
We think that the Gru model was the best. It has the highest testing accuracy, so it is likely to generalize to the problem domain. This approach likely succeeded because it incorporated both ideas of recurrence and embedding, which are appropriate for the task of text classificaiton.

## Bias / Variance Tradeoff
When we trained the models, we noticed the bias/variance tradeoff. The more parameters we gave the model, the better the training accuracy was, but at after a certain point, increasing the size of the model actually worsens the general performance on test data.

